In [42]:
from phasic_tonic.DatasetLoader import DatasetLoader
from phasic_tonic.helper import get_metadata
from phasic_tonic.utils import get_start_end

import numpy as np
import pandas as pd
import pynapple as nap

from tqdm.auto import tqdm
from scipy.io import loadmat

fs_cbd = 2500
fs_os = 2500
fs_rgs = 1000

targetFs = 500
n_down_cbd = fs_cbd/targetFs
n_down_rgs = fs_rgs/targetFs
n_down_os = fs_os/targetFs

CONFIG_DIR = "/home/nero/phasic_tonic/data/dataset_loading.yaml"
OUTPUT_DIR = "/home/nero/phasic_tonic/data/analysis_output/whole_posttrial5/"

Datasets = DatasetLoader(CONFIG_DIR)
mapped_datasets = Datasets.load_datasets()

per_trial_stats = {
    'rat_id': [],
    'study_day': [],
    'condition': [],
    'treatment': [],
    'trial_num': [],
    'total_sleep_duration': [],
    'total_awake_duration': [],
    'total_nrem_duration':[],
    'total_rem_duration':[],
    'counts_wake':[],
    'counts_nrem':[],
    'counts_rem':[]
    }

per_epochs_stats = {
    'rat_id': [],
    'study_day': [],
    'condition': [],
    'treatment': [],
    'trial_num': [],
    'epoch_id' : [],
    'state' : [],
    'duration' : []
}

def per_trial_analysis(metadata, state_epochs):
    for condition in metadata.keys():
        per_trial_stats[condition].append(metadata[condition])

    total_sleep = state_epochs["nrem"].tot_length() + state_epochs["intermediate"].tot_length() + state_epochs["rem"].tot_length()
    
    per_trial_stats['total_sleep_duration'].append(total_sleep)
    per_trial_stats['total_awake_duration'].append(state_epochs["wake"].tot_length())
    per_trial_stats['total_nrem_duration'].append(state_epochs["nrem"].tot_length())
    per_trial_stats['total_rem_duration'].append(state_epochs["rem"].tot_length())

    per_trial_stats['counts_wake'].append(len(state_epochs['wake']))
    per_trial_stats['counts_nrem'].append(len(state_epochs['nrem']))
    per_trial_stats['counts_rem'].append(len(state_epochs['rem']))

def per_epochs_analysis(metadata, state_epochs):    
    for state in ['wake','nrem','rem']:
        for i, dur in enumerate(np.diff(state_epochs[state], 1)):
            for condition in metadata.keys():
                per_epochs_stats[condition].append(metadata[condition])
            per_epochs_stats['state'].append(state)
            per_epochs_stats['duration'].append(dur.item())
            per_epochs_stats['epoch_id'].append(i)

def get_state_epochs(hypno):
    state_epochs = {}
    sleep_states = {1:"wake", 3:"nrem",4:"intermediate",5:"rem"}

    for i in [1, 3, 4, 5]:
        start, end = get_start_end(sleep_states=hypno, sleep_state_id=i)
        state_epochs[sleep_states[i]] = nap.IntervalSet(start=start, end=end)
    return state_epochs


In [38]:
cbd_cnt = 0
rgs_cnt = 0
os_cnt = 0

# Count recordings belonging to CBD dataset
for name in mapped_datasets:
    metadata = get_metadata(name)
    if metadata['treatment'] == 0 or metadata['treatment'] == 1:
        cbd_cnt += 1
    elif metadata['treatment'] == 2 or metadata['treatment'] == 3:
        rgs_cnt += 1
    elif metadata['treatment'] == 4:
        os_cnt += 1

assert cbd_cnt == 170
assert rgs_cnt == 159
assert os_cnt == 210

# Whole post-trial 5

In [43]:
with tqdm(mapped_datasets) as mapped_tqdm:
    for name in mapped_tqdm:
        metadata = get_metadata(name)
        mapped_tqdm.set_postfix_str(name)
        states_fname, _, _ = mapped_datasets[name]
        
        # Load the states
        hypno = loadmat(states_fname)['states'].flatten()

        # Convert each sleep states into dictionary of pynapple IntervalSets
        state_epochs = get_state_epochs(hypno)
        
        per_trial_analysis(metadata, state_epochs)
        per_epochs_analysis(metadata, state_epochs)

df_trials = pd.DataFrame(per_trial_stats)
df_epochs = pd.DataFrame(per_epochs_stats)     

  0%|          | 0/539 [00:00<?, ?it/s]

/home/nero/anaconda3/envs/phasic_tonic/lib/python3.12/site-packages/pynapple/core/interval_set.py:300: UserWarning: Converting IntervalSet to numpy.array
  warnings.warn(
/tmp/ipykernel_79738/1178438583.py:86: UserWarning: Some epochs have no duration
  state_epochs[sleep_states[i]] = nap.IntervalSet(start=start, end=end)


In [44]:
df_trials

,rat_id,study_day,condition,treatment,trial_num,total_sleep_duration,total_awake_duration,total_nrem_duration,total_rem_duration,counts_wake,counts_nrem,counts_rem
0,5,8,HC,0,3,2552.0,136.0,2304.0,201.0,4,4,2
1,5,8,HC,0,1,479.0,2243.0,479.0,0.0,1,1,0
2,5,8,HC,0,2,2549.0,114.0,2090.0,398.0,5,7,4
3,5,8,HC,0,4,2190.0,479.0,1783.0,316.0,6,9,5
4,5,8,HC,0,5,5915.0,4928.0,4679.0,864.0,19,23,13
...,...,...,...,...,...,...,...,...,...,...,...,...
534,11,4,OR,4,1,1435.0,1240.0,1383.0,47.0,9,9,1
535,11,4,OR,4,4,2347.0,325.0,1842.0,412.0,10,9,3
536,11,4,OR,4,2,2025.0,430.0,1727.0,231.0,8,7,3
537,11,4,OR,4,5,6498.0,4230.0,4549.0,1629.0,27,28,19


In [45]:
df_epochs

,rat_id,study_day,condition,treatment,trial_num,epoch_id,state,duration
0,5,8,HC,0,3,0,wake,65.0
1,5,8,HC,0,3,1,wake,31.0
2,5,8,HC,0,3,2,wake,26.0
3,5,8,HC,0,3,3,wake,14.0
4,5,8,HC,0,3,0,nrem,93.0
...,...,...,...,...,...,...,...,...
16045,11,4,OR,4,3,0,rem,10.0
16046,11,4,OR,4,3,1,rem,164.0
16047,11,4,OR,4,3,2,rem,84.0
16048,11,4,OR,4,3,3,rem,137.0


In [36]:
df_trials.to_csv(OUTPUT_DIR+"sleep_states_per_trial.csv", index=False)
df_epochs.to_csv(OUTPUT_DIR+"sleep_states_per_epoch.csv", index=False)

# Segmented post-trial5

In [56]:
per_trial_stats = {
    'rat_id': [],
    'study_day': [],
    'condition': [],
    'treatment': [],
    'trial_num': [],
    'total_sleep_duration': [],
    'total_awake_duration': [],
    'total_nrem_duration':[],
    'total_rem_duration':[],
    'counts_wake':[],
    'counts_nrem':[],
    'counts_rem':[]
    }

per_epochs_stats = {
    'rat_id': [],
    'study_day': [],
    'condition': [],
    'treatment': [],
    'trial_num': [],
    'epoch_id' : [],
    'state' : [],
    'duration' : []
}

OUTPUT_DIR = "/home/nero/phasic_tonic/data/analysis_output/segmented_posttrial5/"

with tqdm(mapped_datasets) as mapped_tqdm:
    for name in mapped_tqdm:
        metadata = get_metadata(name)
        mapped_tqdm.set_postfix_str(name)
        states_fname, _, _ = mapped_datasets[name]
        
        # Load the states
        hypno = loadmat(states_fname)['states'].flatten()
        
        if metadata["trial_num"] == '5':
            for i in range(4):
                metadata["trial_num"] = '5.' + str(i+1)
                hypno_segment = hypno[i*2700:(i+1)*2700]
                state_epochs = get_state_epochs(hypno_segment)

                per_trial_analysis(metadata, state_epochs)
                per_epochs_analysis(metadata, state_epochs)
        else:
            state_epochs = get_state_epochs(hypno)

            per_trial_analysis(metadata, state_epochs)
            per_epochs_analysis(metadata, state_epochs)
            
df_trials = pd.DataFrame(per_trial_stats)
df_epochs = pd.DataFrame(per_epochs_stats)     

  0%|          | 0/539 [00:00<?, ?it/s]

/tmp/ipykernel_79738/1178438583.py:86: UserWarning: Some epochs have no duration
  state_epochs[sleep_states[i]] = nap.IntervalSet(start=start, end=end)


In [57]:
df_trials

,rat_id,study_day,condition,treatment,trial_num,total_sleep_duration,total_awake_duration,total_nrem_duration,total_rem_duration,counts_wake,counts_nrem,counts_rem
0,5,8,HC,0,3,2552.0,136.0,2304.0,201.0,4,4,2
1,5,8,HC,0,1,479.0,2243.0,479.0,0.0,1,1,0
2,5,8,HC,0,2,2549.0,114.0,2090.0,398.0,5,7,4
3,5,8,HC,0,4,2190.0,479.0,1783.0,316.0,6,9,5
4,5,8,HC,0,5.1,2330.0,344.0,1736.0,481.0,5,8,5
...,...,...,...,...,...,...,...,...,...,...,...,...
858,11,4,OR,4,5.1,2141.0,538.0,1494.0,593.0,3,6,5
859,11,4,OR,4,5.2,2450.0,211.0,1720.0,571.0,12,10,8
860,11,4,OR,4,5.3,1264.0,1408.0,845.0,342.0,8,8,5
861,11,4,OR,4,5.4,641.0,2045.0,489.0,122.0,5,5,2


In [58]:
df_epochs

,rat_id,study_day,condition,treatment,trial_num,epoch_id,state,duration
0,5,8,HC,0,3,0,wake,65.0
1,5,8,HC,0,3,1,wake,31.0
2,5,8,HC,0,3,2,wake,26.0
3,5,8,HC,0,3,3,wake,14.0
4,5,8,HC,0,3,0,nrem,93.0
...,...,...,...,...,...,...,...,...
16342,11,4,OR,4,3,0,rem,10.0
16343,11,4,OR,4,3,1,rem,164.0
16344,11,4,OR,4,3,2,rem,84.0
16345,11,4,OR,4,3,3,rem,137.0


In [59]:
df_trials.to_csv(OUTPUT_DIR+"sleep_states_per_trial.csv", index=False)
df_epochs.to_csv(OUTPUT_DIR+"sleep_states_per_epoch.csv", index=False)